# ngrams

> Functionality for ngram analysis.
- toc: false
- page-layout: full

In [ ]:
#| default_exp ngrams

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
import numpy as np
import time
import polars as pl
from spacy.attrs import ORTH, LOWER # remove? - add ENT_TYPE, ENT_IOB
import math
from fastcore.basics import patch

In [ ]:
#| export
from conc.corpus import Corpus
from conc.result import Result
from conc.core import logger, PAGE_SIZE

In [ ]:
#| hide
from conc.core import set_logger_state

In [ ]:
#| hide
import os

In [ ]:
#| hide
source_path = f'{os.environ.get("HOME")}/data/'
save_path = f'{os.environ.get("HOME")}/data/conc-test-corpora/'

path_to_toy_corpus = f'{save_path}toy.corpus'
path_to_brown_corpus = f'{save_path}brown.corpus'
path_to_reuters_corpus = f'{save_path}reuters.corpus'

## Using the Ngrams class

There are examples below showing how to use the Ngrams class directly to output Ngram clusters or frequency lists based on ngrams. The recommended way to use this functionality is through the [Conc](https://geoffford.nz/conc/api/conc.html) class. This provides an interface to create frequency lists, concordances, collocation tables, keyword tables and more. 

## Ngrams class API reference

In [ ]:
#| export
class Ngrams:
	""" Class for n-gram analysis reporting. """
	def __init__(self,
			  corpus:Corpus # Corpus instance
			  ): 

		if type(corpus) != Corpus:
			raise ValueError('Conc ngrams functionality is only available for instances of the Corpus class.')

		self.corpus = corpus


In [ ]:
#| hide
from conc.corpora import show_toy_corpus

In [ ]:
#| hide
toy = Corpus().load(path_to_toy_corpus)
ngrams_toy = Ngrams(toy)

In [ ]:
#| exporti
@patch
def _get_ngrams(self:Ngrams, 
			   token_sequence: list[np.ndarray], # token sequence to get index for 
			   index_id: int, # index to search (i.e. ORTH, LOWER)
			   token_positions: list[np.ndarray], # positions of token sequence, returned by get_token_positions 
			   ngram_length: int = 2, # length of ngram
			   ngram_token_position: str = 'LEFT', # specify if token sequence is on LEFT or RIGHT (support for ngrams with token in middle of sequence is in-development))
			   exclude_punctuation:bool=False # exclude ngrams with tokens
			   ) -> tuple[np.ndarray, list[np.ndarray]]: # array of ngrams results, revised token positions

	""" Get ngram data for a token sequence. """
	
	start_time = time.time()
	sequence_len = len(token_sequence[0])
	variants_len = len(token_sequence)
	token_index_len = len(token_positions[0])

	if index_id == ORTH:
		index = 'orth_index'
	else:
		index = 'lower_index'

	if ngram_token_position == 'RIGHT':
		ngrams = self.corpus.get_tokens_in_context(token_positions = token_positions, index = index, context_length = ngram_length, position_offset = 0, position_offset_step = -1, exclude_punctuation = False, convert_eof = False)
		ngrams = ngrams[::-1, :] # reversing order as retrieved right to left
	elif ngram_token_position == 'LEFT':
		ngrams = self.corpus.get_tokens_in_context(token_positions = token_positions, index = index, context_length = ngram_length, position_offset = 0, position_offset_step = 1, exclude_punctuation = False, convert_eof = False)
	elif ngram_token_position == 'MIDDLE':
		length_per_side = math.ceil((ngram_length - sequence_len) / 2)
		if length_per_side * 2 + sequence_len != ngram_length:
			logger.warning(f'Ngram length {ngram_length} does not make sense with the provided tokens and ngram_token_position MIDDLE. Adjusting ngram_length.')

		logger.debug(f'Tokens per side for middle token position: {length_per_side}, with ngram length {ngram_length} and sequence length {sequence_len}')
		right_tokens= self.corpus.get_tokens_in_context(token_positions = token_positions, index = index, context_length = sequence_len + length_per_side, position_offset = 0, position_offset_step = 1, exclude_punctuation = False, convert_eof = False)
		left_tokens = self.corpus.get_tokens_in_context(token_positions = token_positions, index = index, context_length = length_per_side, position_offset = -1, position_offset_step = -1, exclude_punctuation = False, convert_eof = False)
		left_tokens = left_tokens[::-1, :] # reversing order as retrieved right to left
		# combining so that has shape (ngram_length, len(token_positions[0]))
		ngrams = np.concatenate((left_tokens, right_tokens), axis=0)

	logger.debug(f'Ngrams shape after retrieval {ngrams.shape}')

	# elif ngram_token_position == 'MIDDLE': # further development needed - in roadmap
	# 	ngram_range = range(-1 * ngram_length + sequence_len + 1, sequence_len + 1)

	# old retrieval method - aligning with collocates context retrieval
	# ngrams = []
	
	# for pos in ngram_range:
	# 	if variants_len == 1 and pos > -1 and pos < sequence_len:
	# 		ngrams.append(np.full(token_index_len, token_sequence[0][pos]))
	# 	else:
	# 		seq = token_positions[0] + pos
	# 		ngrams.append(self.corpus.get_tokens_by_index(index)[seq])

	# ngrams = np.stack(ngrams)
	# logger.info(f'Ngrams ({ngrams.shape}) retrieval time: {(time.time() - start_time):.5f} seconds')

	# getting positions to search for EOF_TOKEN and filter out ngrams crossing doc boundaries
	#positions = (np.array(ngram_range)[:, None] != np.arange(sequence_len)).all(axis=1)
	# ngrams = np.delete(ngrams, np.where(ngrams[positions] == self.corpus.EOF_TOKEN)[1], axis=1)
	filter = [self.corpus.EOF_TOKEN]

	if exclude_punctuation: # see above - ngrams returned with punctuation tokens - and then cleaned if exclude_punctuation is True
		filter.extend(self.corpus.punct_tokens)
	
	mask = np.isin(ngrams, filter)
	ngrams = np.delete(ngrams, np.where(mask)[1], axis=1)
	logger.debug(f'Ngrams shape after filter {ngrams.shape}')

	token_positions[0] = np.delete(token_positions[0], np.where(mask)[1], axis=0)
	logger.debug(f'Token positions after filter {token_positions[0].shape}')

	logger.info(f'Ngrams ({ngrams.shape[1]}) retrieval time: {(time.time() - start_time):.5f} seconds')
	return ngrams, token_positions


In [ ]:
#| hide
show_toy_corpus(f'{source_path}toy.csv')
set_logger_state('verbose')
token_str = 'the'
token_sequence, index_id = toy.tokenize(token_str, simple_indexing=True)
token_positions = toy.get_token_positions(token_sequence, index_id)
print(token_positions)
ngrams, token_positions = ngrams_toy._get_ngrams(token_sequence, index_id, token_positions, ngram_length = 4, ngram_token_position = 'LEFT')
print(token_positions)
set_logger_state('quiet')

source,text,category,species
1.txt,The cat sat on the mat.,feline,cat
2.txt,The dog sat on the mat.,canine,dog
3.txt,The cat is meowing.,feline,cat
4.txt,The dog is barking.,canine,dog
5.txt,The cat is climbing a tree.,feline,cat
6.txt,The dog is digging a hole.,canine,dog


2025-07-07 10:46:51 - INFO - _init_token_arrays - Created tokens_array in 0.002 seconds
2025-07-07 10:46:51 - INFO - _init_token_arrays - Created tokens_lookup in 0.000 seconds
2025-07-07 10:46:51 - INFO - _init_token_arrays - Created tokens_sort_order in 0.001 seconds
2025-07-07 10:46:51 - INFO - tokenize - Tokenization time: 0.00570 seconds
2025-07-07 10:46:51 - DEBUG - get_tokens_by_index - Getting tokens for index: lower_index
2025-07-07 10:46:51 - INFO - get_tokens_by_index - Got tokens for index lower_index with exclude_punctuation False in 0.001 seconds
2025-07-07 10:46:51 - INFO - get_token_positions - Token indexing (8) time: 0.00229 seconds
2025-07-07 10:46:51 - DEBUG - get_tokens_by_index - Getting tokens for index: lower_index
2025-07-07 10:46:51 - INFO - get_tokens_by_index - Tokens for index lower_index with exclude_punctuation False already cached, returning cached result in 0.000 seconds
2025-07-07 10:46:51 - DEBUG - get_tokens_by_index - Getting tokens for index: lower

[array([100, 104, 108, 112, 116, 122, 128, 136])]
[array([100, 108, 116, 122, 128, 136])]


In [ ]:
#| hide
token_str = 'dog'
token_sequence, index_id = toy.tokenize(token_str, simple_indexing=True)
token_positions = toy.get_token_positions(token_sequence, index_id)
ngrams, token_positions = ngrams_toy._get_ngrams(token_sequence, index_id, token_positions, ngram_length = 2, ngram_token_position = 'LEFT')
assert ngrams.shape[0] == 2
assert ngrams.shape[1] == 3
assert tuple(toy.token_ids_to_tokens(ngrams[0])) == tuple(['dog', 'dog', 'dog'])
assert tuple(toy.token_ids_to_tokens(ngrams[1])) == tuple(['sat', 'is', 'is'])

# testing right
ngrams, token_positions = ngrams_toy._get_ngrams(token_sequence, index_id, token_positions, ngram_length = 2, ngram_token_position = 'RIGHT')
assert ngrams.shape[0] == 2
assert ngrams.shape[1] == 3
assert tuple(toy.token_ids_to_tokens(ngrams[1])) == tuple(['dog', 'dog', 'dog'])
assert tuple(toy.token_ids_to_tokens(ngrams[0])) == tuple(['the', 'the', 'the'])

In [ ]:
#| hide
token_str = 'the'
token_sequence, index_id = toy.tokenize(token_str, simple_indexing=True)
token_positions = toy.get_token_positions(token_sequence, index_id)
ngrams, token_positions = ngrams_toy._get_ngrams(token_sequence, index_id, token_positions, ngram_length = 3, ngram_token_position = 'LEFT', exclude_punctuation = False)
assert ngrams.shape[0] == 3
assert ngrams.shape[1] == 8
assert token_positions[0].shape[0] == 8
assert tuple(toy.token_ids_to_tokens(ngrams[0])) == tuple(['the', 'the', 'the', 'the', 'the', 'the', 'the', 'the'])
assert tuple(toy.token_ids_to_tokens(ngrams[1])) == tuple(['cat', 'mat', 'dog', 'mat', 'cat', 'dog', 'cat', 'dog'])
assert tuple(toy.token_ids_to_tokens(ngrams[2])) == tuple(['sat', '.', 'sat', '.', 'is', 'is', 'is', 'is'])

ngrams, token_positions = ngrams_toy._get_ngrams(token_sequence, index_id, token_positions, ngram_length = 3, ngram_token_position = 'LEFT', exclude_punctuation = True)
assert ngrams.shape[0] == 3
assert ngrams.shape[1] == 6
assert token_positions[0].shape[0] == 6
assert tuple(toy.token_ids_to_tokens(ngrams[0])) == tuple(['the', 'the',  'the', 'the', 'the', 'the'])
assert tuple(toy.token_ids_to_tokens(ngrams[1])) == tuple(['cat', 'dog',  'cat', 'dog', 'cat', 'dog'])
assert tuple(toy.token_ids_to_tokens(ngrams[2])) == tuple(['sat', 'sat', 'is', 'is', 'is', 'is'])

# case where punctuation can be included, but triggering EOF tokens - two not included
ngrams, token_positions = ngrams_toy._get_ngrams(token_sequence, index_id, token_positions, ngram_length = 4, ngram_token_position = 'LEFT', exclude_punctuation = False)
assert ngrams.shape[0] == 4
assert ngrams.shape[1] == 6
assert tuple(toy.token_ids_to_tokens(ngrams[0])) == tuple(['the', 'the',  'the', 'the', 'the', 'the'])
assert tuple(toy.token_ids_to_tokens(ngrams[1])) == tuple(['cat', 'dog',  'cat', 'dog', 'cat', 'dog'])
assert tuple(toy.token_ids_to_tokens(ngrams[2])) == tuple(['sat', 'sat', 'is', 'is', 'is', 'is'])
assert tuple(toy.token_ids_to_tokens(ngrams[3])) == tuple(['on', 'on', 'meowing', 'barking', 'climbing', 'digging'])

token_str = 'the dog'
token_sequence, index_id = toy.tokenize(token_str, simple_indexing=True)
token_positions = toy.get_token_positions(token_sequence, index_id)
ngrams, token_positions = ngrams_toy._get_ngrams(token_sequence, index_id, token_positions, ngram_length = 3, ngram_token_position = 'LEFT', exclude_punctuation = False)
assert ngrams.shape[0] == 3
assert ngrams.shape[1] == 3
assert tuple(toy.token_ids_to_tokens(ngrams[0])) == tuple(['the', 'the',  'the'])
assert tuple(toy.token_ids_to_tokens(ngrams[1])) == tuple(['dog', 'dog', 'dog'])
assert tuple(toy.token_ids_to_tokens(ngrams[2])) == tuple(['sat', 'is', 'is'])

In [ ]:
#| hide
# set_logger_state('verbose')
# brown = Corpus().load(path_to_brown_corpus)
# ngrams_brown = Ngrams(brown)
# token_str = 'The'
# token_sequence, index_id = brown.tokenize(token_str, simple_indexing=True)
# token_positions = brown.get_token_positions(token_sequence, index_id)
# ngrams = ngrams_brown._get_ngrams(token_sequence, index_id, token_positions, ngram_length = 3, ngram_token_position = 'RIGHT', exclude_punctuation = False)
# print(ngrams.shape)
# # find any ngrams[0] that contain brown.space_tokens
# from_x = 0
# to_x = 100
# df = pl.DataFrame([brown.token_ids_to_tokens(ngrams[0]),
#     brown.token_ids_to_tokens(ngrams[1]), brown.token_ids_to_tokens(ngrams[2])])
# display(df.with_row_index('row').filter((pl.col('column_1') == '\n') & 
#     (pl.col('row') >= from_x) & (pl.col('row') <= to_x)
# ).head(20))

# ngrams = ngrams_brown._get_ngrams(token_sequence, index_id, token_positions, ngram_length = 2, ngram_token_position = 'RIGHT', exclude_punctuation = False)
# print(ngrams.shape)
# from_x = 0
# to_x = 100
# df = pl.DataFrame([brown.token_ids_to_tokens(ngrams[0]),
#     brown.token_ids_to_tokens(ngrams[1])])
# display(df.with_row_index('row').filter( 
#     (pl.col('row') == 37) | (pl.col('row') == 78) | (pl.col('row') == 94)).head(20))
# set_logger_state('quiet')

In [ ]:
#| hide
# token_str = 'ocean'
# token_sequence, index_id = reuters.tokenize(token_str, simple_indexing=True)
# token_positions = reuters.get_token_positions(token_sequence, index_id)
# ngrams = ngrams_reuters._get_ngrams(token_sequence, index_id, token_positions, ngram_length = 2, ngram_token_position = 'LEFT')
# print(ngrams)

In [ ]:
#| export
@patch
def ngrams(self: Ngrams, 
		   token_str: str, # token string to get ngrams for 
		   ngram_length:int|None = 2, # length of ngram, if set to None it will use the number of tokens in the token_str + 1
		   ngram_token_position: str = 'LEFT', # specify if token sequence is on LEFT or RIGHT or MIDDLE (support for other positions is in-development)
		   normalize_by:int=10000, # normalize frequencies by a number (e.g. 10000)
		   page_size:int = PAGE_SIZE, # number of results to display per results page 
		   page_current:int = 1, # current page of results
		   show_all_columns:bool = False, # return raw df with all columns or just ngram and frequency
		   exclude_punctuation:bool=True, # do not return ngrams with punctuation tokens
		   use_cache:bool = True # retrieve the results from cache if available (currently ignored)
		   ) -> Result: # return a Result object with ngram data
	""" Report ngram frequencies containing a token string. """

	if type(normalize_by) != int:
		raise ValueError('normalize_by must be an integer, e.g. 1000000 or 10000')

	token_sequence, index_id = self.corpus.tokenize(token_str, simple_indexing=True)

	if ngram_length is None:
		if ngram_token_position == 'MIDDLE':
			ngram_length = len(token_sequence[0]) + 2
		else:
			ngram_length = len(token_sequence[0]) + 1

	start_time = time.time()
	use_cache = False
	cache_id = tuple(['ngram'] + list(token_sequence) + [ngram_length, ngram_token_position]) # before reenabling will need to make sure the cache_id matches options above (e.g. could get differences based on exclude punctuation etc but cache first currently)

	if use_cache == True and cache_id in self.corpus.results_cache:
		logger.info('Using cached ngrams results')
		ngrams_report = self.corpus.results_cache[cache_id][0]
		total_unique = self.corpus.results_cache[cache_id][1]
		total_count = self.corpus.results_cache[cache_id][2]
	else:
		token_positions = self.corpus.get_token_positions(token_sequence, index_id)
		
		if len(token_positions[0]) == 0:
			logger.info('No tokens found')
			return Result(type = 'ngrams', df=pl.DataFrame(), title=f'Ngrams for "{token_str}"', description=f'No matches', summary_data={}, formatted_data=[])

		logger.info('Retrieving ngrams results')
		ngrams, token_positions = self._get_ngrams(token_sequence, index_id, token_positions, ngram_length = ngram_length, ngram_token_position = ngram_token_position, exclude_punctuation=exclude_punctuation)
		total_count = ngrams.shape[1]
		if ngram_token_position == 'MIDDLE':
			expected_ngram_length = len(token_sequence[0]) +  (2 * (math.ceil((ngram_length - len(token_sequence[0])) / 2)))
			if expected_ngram_length != ngram_length:
				logger.warning(f'Adjusting ngram length from {ngram_length} to {expected_ngram_length}')
				ngram_length = expected_ngram_length
		schema = [f'token_{i+1}' for i in range(ngram_length)]
		ngrams_report = pl.DataFrame(ngrams.T, schema=schema).to_struct(name = 'ngram_token_ids').value_counts(sort=True).rename({"count": "frequency"}) # TODO - potentially improve by adding second sort so frequency lists are stable

		# if show_document_frequency:
			# to be added in with # show_document_frequency:bool = False, # return document frequency in output
		# 	ngrams_report = ngrams_report.with_columns(pl.Series(name='doc_id', values=self.corpus.get_tokens_by_index('token2doc_index')[token_positions[0]]))

		ngrams_report = ngrams_report.with_row_index(name='rank', offset=1)
		total_unique = len(ngrams_report)

		if use_cache == True:
			self.corpus.results_cache[cache_id] = (ngrams_report, total_unique, total_count)
	
	count_tokens, tokens_descriptor, total_descriptor = self.corpus.get_token_count_text(exclude_punctuation)

	resultset_start = page_size*(page_current-1)

	# get specific chunk of report into polars based on resultset_start:
	ngrams_report_page = ngrams_report.slice(resultset_start, page_size).unnest('ngram_token_ids')
	ngrams_report_page = ngrams_report_page.with_columns(((pl.col("frequency") / pl.lit(count_tokens)) * normalize_by).alias('normalized_frequency'))

	token_strs = []
	for i in range(ngram_length):
		token_strs.append(self.corpus.token_ids_to_tokens(ngrams_report_page[f'token_{i+1}'].to_numpy()))
	token_strs = np.array(token_strs)
	ngram_text = [' '.join(column) for column in token_strs.T]
	ngrams_report_page = ngrams_report_page.with_columns(pl.Series(name="ngram", values=ngram_text))

	total_pages = math.ceil(total_unique/page_size)
	summary_data = {'ngram_length': ngram_length, 'ngram_token_position': ngram_token_position, 'total_unique': total_unique, 'total_count': total_count, 'page_current': page_current, 'total_pages': total_pages}
	formatted_data = [f'Report based on {tokens_descriptor}', f'Ngram length: {ngram_length}, Token position: {ngram_token_position.lower()}']

	if exclude_punctuation:
		formatted_data.append(f'Ngrams containing punctuation tokens excluded')

	if normalize_by is not None:
		formatted_data.append(f'Normalized Frequency is per {normalize_by:,.0f} tokens')

	formatted_data.extend([f'Total unique ngrams: {total_unique:,}', f'Total ngrams: {total_count:,}'])

	if page_size != 0 and total_count > page_size:
		formatted_data.extend([f'Showing {min(page_size, total_count)} rows', f'Page {page_current} of {total_pages}']) 

	if show_all_columns == False:
		ngrams_report_page = ngrams_report_page[['rank', 'ngram', 'frequency', 'normalized_frequency']]
	
	logger.info(f'Ngrams report time: {(time.time() - start_time):.5f} seconds')

	return Result(type = 'ngrams', df=ngrams_report_page, title=f'Ngrams for "{token_str}"', description=f'{self.corpus.name}', summary_data=summary_data, formatted_data=formatted_data)


In [ ]:
#| hide
show_toy_corpus(f'{source_path}toy.csv')
token_str = 'the'
test_result = ngrams_toy.ngrams(token_str).to_frame()
assert test_result.filter(pl.col('ngram') == 'the cat').select('frequency').item() == 3
assert test_result.filter(pl.col('ngram') == 'the mat').select('frequency').item() == 2
assert test_result.filter(pl.col('ngram') == 'the dog').select('frequency').item() == 3
assert test_result.select(pl.len()).item() == 3

source,text,category,species
1.txt,The cat sat on the mat.,feline,cat
2.txt,The dog sat on the mat.,canine,dog
3.txt,The cat is meowing.,feline,cat
4.txt,The dog is barking.,canine,dog
5.txt,The cat is climbing a tree.,feline,cat
6.txt,The dog is digging a hole.,canine,dog


In [ ]:
#| hide
show_toy_corpus(f'{source_path}toy.csv')
token_str = 'the dog'
test_result = ngrams_toy.ngrams(token_str, ngram_length = None).to_frame()
assert test_result.filter(pl.col('ngram') == 'the dog sat').select('frequency').item() == 1
assert test_result.filter(pl.col('ngram') == 'the dog is').select('frequency').item() == 2
assert test_result.select(pl.len()).item() == 2

source,text,category,species
1.txt,The cat sat on the mat.,feline,cat
2.txt,The dog sat on the mat.,canine,dog
3.txt,The cat is meowing.,feline,cat
4.txt,The dog is barking.,canine,dog
5.txt,The cat is climbing a tree.,feline,cat
6.txt,The dog is digging a hole.,canine,dog


In [ ]:
#| hide
set_logger_state('verbose')
token_str = 'is'
display(ngrams_toy.ngrams(token_str, ngram_length = 2, ngram_token_position='MIDDLE').to_frame())
set_logger_state('quiet')

2025-07-07 10:46:51 - INFO - tokenize - Tokenization time: 0.00013 seconds
2025-07-07 10:46:51 - INFO - get_token_positions - Token indexing (4) time: 0.00007 seconds
2025-07-07 10:46:51 - INFO - ngrams - Retrieving ngrams results
2025-07-07 10:46:51 - WARNING - _get_ngrams - Ngram length 2 does not make sense with the provided tokens and ngram_token_position MIDDLE. Adjusting ngram_length.
2025-07-07 10:46:51 - DEBUG - _get_ngrams - Tokens per side for middle token position: 1, with ngram length 2 and sequence length 1
2025-07-07 10:46:51 - DEBUG - get_tokens_by_index - Getting tokens for index: lower_index
2025-07-07 10:46:51 - INFO - get_tokens_by_index - Tokens for index lower_index with exclude_punctuation False already cached, returning cached result in 0.000 seconds
2025-07-07 10:46:51 - DEBUG - get_tokens_by_index - Getting tokens for index: lower_index
2025-07-07 10:46:51 - INFO - get_tokens_by_index - Tokens for index lower_index with exclude_punctuation False already cached,

rank,ngram,frequency,normalized_frequency
1,"""cat is meowing""",1,312.5
2,"""dog is barking""",1,312.5
3,"""cat is climbing""",1,312.5
4,"""dog is digging""",1,312.5


#### Examples

See the note above about accessing this functionality through the [Conc](https://geoffford.nz/conc/api/conc.html) class.

In [ ]:
# load the corpus
reuters = Corpus().load(path_to_reuters_corpus)

# instantiate the Ngrams class
ngrams_reuters = Ngrams(reuters)

In [ ]:
# run the ngrams method and display the results
ngrams_reuters.ngrams('environmental', ngram_length = 2, ngram_token_position = 'LEFT').display()

Ngrams for "environmental" 
 
 
 Reuters Corpus 
 
 
 Rank 
 Ngram 
 Frequency 
 Normalized Frequency 
 
 
 
 
 1 
 environmental protection 
 4 
 0.03 
 
 
 2 
 environmental systems 
 4 
 0.03 
 
 
 3 
 environmental services 
 3 
 0.02 
 
 
 4 
 environmental damage 
 2 
 0.01 
 
 
 5 
 environmental regulations 
 2 
 0.01 
 
 
 6 
 environmental impact 
 2 
 0.01 
 
 
 7 
 environmental controls 
 1 
 0.01 
 
 
 8 
 environmental approval 
 1 
 0.01 
 
 
 9 
 environmental and 
 1 
 0.01 
 
 
 10 
 environmental sciences 
 1 
 0.01 
 
 
 11 
 environmental service 
 1 
 0.01 
 
 
 12 
 environmental concerns 
 1 
 0.01 
 
 
 13 
 environmental issues 
 1 
 0.01 
 
 
 14 
 environmental power 
 1 
 0.01 
 
 
 15 
 environmental plan 
 1 
 0.01 
 
 
 16 
 environmental had 
 1 
 0.01 
 
 
 17 
 environmental control 
 1 
 0.01 
 
 
 18 
 environmental management 
 1 
 0.01 
 
 
 19 
 environmental subsidiary 
 1 
 0.01 
 
 
 20 
 environmental was 
 1 
 0.01 
 
 
 
 
 
 Report based on word tokens 
 


 
 Ngram length: 2, Token position: left 
 


 
 Ngrams containing punctuation tokens excluded 
 


 
 Normalized Frequency is per 10,000 tokens 
 


 
 Total unique ngrams: 21 
 


 
 Total ngrams: 32 
 


 
 Showing 20 rows 
 


 
 Page 1 of 2

In [ ]:
# run the ngrams method and display the results
ngrams_reuters.ngrams('the highest', ngram_length = 3, ngram_token_position = 'LEFT', page_size = 10).display()

Ngrams for "the highest" 
 
 
 Reuters Corpus 
 
 
 Rank 
 Ngram 
 Frequency 
 Normalized Frequency 
 
 
 
 
 1 
 the highest since 
 8 
 0.06 
 
 
 2 
 the highest level 
 4 
 0.03 
 
 
 3 
 the highest in 
 3 
 0.02 
 
 
 4 
 the highest rate 
 2 
 0.01 
 
 
 5 
 the highest interest 
 2 
 0.01 
 
 
 6 
 the highest priority 
 2 
 0.01 
 
 
 7 
 the highest number 
 2 
 0.01 
 
 
 8 
 the highest agriculture 
 2 
 0.01 
 
 
 9 
 the highest such 
 2 
 0.01 
 
 
 10 
 the highest positive 
 2 
 0.01 
 
 
 
 
 
 Report based on word tokens 
 


 
 Ngram length: 3, Token position: left 
 


 
 Ngrams containing punctuation tokens excluded 
 


 
 Normalized Frequency is per 10,000 tokens 
 


 
 Total unique ngrams: 31 
 


 
 Total ngrams: 50 
 


 
 Showing 10 rows 
 


 
 Page 1 of 4

In [ ]:
#| export
@patch
def ngram_frequencies(self: Ngrams, 
                ngram_length:int=2, # length of ngram
                case_sensitive:bool=False, # frequencies for tokens lowercased or with case preserved
				normalize_by:int=10000, # normalize frequencies by a number (e.g. 10000)
				page_size:int=PAGE_SIZE, # number of rows to return
				page_current:int=1, # current page
				show_document_frequency:bool=False, #show document frequency in output
				exclude_punctuation:bool=True # exclude ngrams containing punctuation tokens
				) -> Result: # return a Result object with the frequency table
	""" Report frequent ngrams. """
	
	if type(normalize_by) != int:
		raise ValueError('normalize_by must be an integer, e.g. 1000000 or 10000')

	start_time = time.time()

	if case_sensitive:
		index = 'orth_index'
	else:
		index = 'lower_index'

	filter = [self.corpus.EOF_TOKEN]
	if exclude_punctuation == True:
		filter += self.corpus.punct_tokens

	resultset_start = page_size*(page_current-1)

	count_tokens, tokens_descriptor, total_descriptor = self.corpus.get_token_count_text(exclude_punctuation)
	formatted_data = [f'Report based on {tokens_descriptor}']
	formatted_data.append(f'Ngram length: {ngram_length}')

	if exclude_punctuation:
		formatted_data.append(f'Ngrams containing punctuation tokens excluded')

	ngrams = self.corpus.tokens.select(pl.col('token2doc_index'), pl.col(index).alias('token_1')).with_row_index('position')

	ngrams = ngrams.with_columns([pl.col('token_1').shift(-i).alias(f'token_{i+1}') for i in range(1, ngram_length)])

	schema = [f'token_{i+1}' for i in range(ngram_length)]

	ngrams_report = ngrams.group_by(schema).agg(pl.len().alias("frequency")).sort(by="frequency", descending=True) # TODO - potentially improve by adding second sort so frequency lists are stable

	if show_document_frequency:
		ngrams_report = ngrams_report.join(
			ngrams.group_by(schema).agg(pl.col('token2doc_index').n_unique().alias('document_frequency')), on = schema, how='left', maintain_order='left'
		)

	for i in range(ngram_length):
		ngrams_report = ngrams_report.filter(~pl.col(f'token_{i+1}').is_in(filter))

	total_unique = ngrams_report.select(pl.len()).collect().item()
	total_count = ngrams_report.select(pl.col('frequency').sum()).collect().item()

	ngrams_report_page = ngrams_report.slice(resultset_start, page_size).collect(engine = 'streaming')
	logger.info(f'collected report page: {(time.time() - start_time):.5f} seconds')
	token_strs = []
	for i in range(ngram_length):
		token_strs.append(self.corpus.token_ids_to_tokens(ngrams_report_page.select(pl.col(f'token_{i+1}')).to_numpy().flatten()))
	token_strs = np.array(token_strs)
	ngram_text = [' '.join(column) for column in token_strs.T]
	ngrams_report_page = ngrams_report_page.with_columns(pl.Series(name="ngram", values=ngram_text)).with_row_index(name='rank', offset=(page_current-1)*page_size+1)
	ngrams_report_page = ngrams_report_page.with_columns(((pl.col("frequency") / pl.lit(count_tokens)) * normalize_by).alias('normalized_frequency'))
	formatted_data.append(f'Normalized Frequency is per {normalize_by:,.0f} tokens')

	formatted_data.extend([f'Total unique ngrams: {total_unique:,}', f'Total ngrams: {total_count:,}'])

	total_pages = math.ceil(total_unique/page_size)
	if page_size != 0 and total_count > page_size:
		formatted_data.extend([f'Showing {min(page_size, total_count)} rows', f'Page {page_current} of {total_pages}']) 

	columns = ['rank', 'ngram', 'frequency', 'normalized_frequency']
	if show_document_frequency:
		columns.append('document_frequency')
		
	ngrams_report_page = ngrams_report_page[columns]

	return Result(type = 'ngram_frequencies', df=ngrams_report_page, title=f'Ngram Frequencies', description=f'{self.corpus.name}', summary_data = {}, formatted_data = formatted_data)

Ngram frequencies is the slowest operation in Conc currently and will be optimised in the future. 

#### Examples

See the note above about accessing this functionality through the [Conc](https://geoffford.nz/conc/api/conc.html) class.

In [ ]:
#| hide
# ngrams_report = brown.tokens.select(pl.col('lower_index').alias('token_1')).with_row_index('position').collect()
# ngrams_report = ngrams_report.join(brown.spaces.collect().select(pl.col('position').alias('space_position'), pl.lit(True).alias('is_space')), left_on='position', right_on='space_position', how='left', maintain_order='left')
# print(ngrams_report.filter(pl.col('position')>= 100).head(10))
# ngrams_report = ngrams_report.filter(pl.col('is_space').is_null())
# print(ngrams_report.shape)
# print(ngrams_report.filter(pl.col('position')>= 99).head(10))

In [ ]:
#| hide
show_toy_corpus(f'{source_path}toy.csv')
test_result = ngrams_toy.ngram_frequencies(ngram_length = 2, case_sensitive = False, exclude_punctuation = False, page_current = 1, page_size = 100).to_frame()
assert test_result.select(pl.col('frequency').sum()).item() == 32
test_result = ngrams_toy.ngram_frequencies(ngram_length = 2, case_sensitive = False, exclude_punctuation = True, page_current = 1, page_size = 100).to_frame()
assert test_result.select(pl.col('frequency').sum()).item() == 26

source,text,category,species
1.txt,The cat sat on the mat.,feline,cat
2.txt,The dog sat on the mat.,canine,dog
3.txt,The cat is meowing.,feline,cat
4.txt,The dog is barking.,canine,dog
5.txt,The cat is climbing a tree.,feline,cat
6.txt,The dog is digging a hole.,canine,dog


In [ ]:
#| hide
ngrams_reuters.ngrams('the company', ngram_length = 3).display()

Ngrams for "the company" 
 
 
 Reuters Corpus 
 
 
 Rank 
 Ngram 
 Frequency 
 Normalized Frequency 
 
 
 
 
 1 
 the company said 
 1,173 
 8.39 
 
 
 2 
 the company 's 
 518 
 3.70 
 
 
 3 
 the company also 
 107 
 0.76 
 
 
 4 
 the company has 
 69 
 0.49 
 
 
 5 
 the company to 
 69 
 0.49 
 
 
 6 
 the company is 
 59 
 0.42 
 
 
 7 
 the company reported 
 51 
 0.36 
 
 
 8 
 the company will 
 49 
 0.35 
 
 
 9 
 the company had 
 47 
 0.34 
 
 
 10 
 the company was 
 46 
 0.33 
 
 
 11 
 the company would 
 43 
 0.31 
 
 
 12 
 the company and 
 32 
 0.23 
 
 
 13 
 the company earned 
 22 
 0.16 
 
 
 14 
 the company did 
 20 
 0.14 
 
 
 15 
 the company for 
 16 
 0.11 
 
 
 16 
 the company today 
 15 
 0.11 
 
 
 17 
 the company in 
 14 
 0.10 
 
 
 18 
 the company added 
 13 
 0.09 
 
 
 19 
 the company last 
 11 
 0.08 
 
 
 20 
 the company at 
 11 
 0.08 
 
 
 
 
 
 Report based on word tokens 
 


 
 Ngram length: 3, Token position: left 
 


 
 Ngrams containing punctuation tokens excluded 
 


 
 Normalized Frequency is per 10,000 tokens 
 


 
 Total unique ngrams: 221 
 


 
 Total ngrams: 2,837 
 


 
 Showing 20 rows 
 


 
 Page 1 of 12

In [ ]:
ngrams_reuters.ngram_frequencies(ngram_length = 3, case_sensitive = False).display()

Ngram Frequencies 
 
 
 Reuters Corpus 
 
 
 Rank 
 Ngram 
 Frequency 
 Normalized Frequency 
 
 
 
 
 1 
 the company said 
 1,173 
 8.39 
 
 
 2 
 mln dlrs in 
 795 
 5.68 
 
 
 3 
 cts vs loss 
 665 
 4.75 
 
 
 4 
 said it has 
 636 
 4.55 
 
 
 5 
 mln avg shrs 
 620 
 4.43 
 
 
 6 
 pct of the 
 608 
 4.35 
 
 
 7 
 the united states 
 603 
 4.31 
 
 
 8 
 qtr net shr 
 574 
 4.10 
 
 
 9 
 dlrs a share 
 546 
 3.90 
 
 
 10 
 inc said it 
 523 
 3.74 
 
 
 11 
 the company 's 
 518 
 3.70 
 
 
 12 
 cts net loss 
 517 
 3.70 
 
 
 13 
 the end of 
 501 
 3.58 
 
 
 14 
 cts a share 
 494 
 3.53 
 
 
 15 
 is expected to 
 429 
 3.07 
 
 
 16 
 corp said it 
 412 
 2.95 
 
 
 17 
 nine mths shr 
 412 
 2.95 
 
 
 18 
 said in a 
 407 
 2.91 
 
 
 19 
 the bank of 
 380 
 2.72 
 
 
 20 
 billion dlrs in 
 373 
 2.67 
 
 
 
 
 
 Report based on word tokens 
 


 
 Ngram length: 3 
 


 
 Ngrams containing punctuation tokens excluded 
 


 
 Normalized Frequency is per 10,000 tokens 
 


 
 Total unique ngrams: 684,778 
 


 
 Total ngrams: 1,128,352 
 


 
 Showing 20 rows 
 


 
 Page 1 of 34239

In [ ]:
ngrams_reuters.ngram_frequencies(ngram_length = 3, case_sensitive = True).display()

Ngram Frequencies 
 
 
 Reuters Corpus 
 
 
 Rank 
 Ngram 
 Frequency 
 Normalized Frequency 
 
 
 
 
 1 
 The company said 
 747 
 5.34 
 
 
 2 
 mln dlrs in 
 726 
 5.19 
 
 
 3 
 cts vs loss 
 645 
 4.61 
 
 
 4 
 said it has 
 632 
 4.52 
 
 
 5 
 mln Avg shrs 
 615 
 4.40 
 
 
 6 
 pct of the 
 608 
 4.35 
 
 
 7 
 QTR NET Shr 
 559 
 4.00 
 
 
 8 
 the United States 
 524 
 3.75 
 
 
 9 
 dlrs a share 
 519 
 3.71 
 
 
 10 
 Inc said it 
 514 
 3.67 
 
 
 11 
 cts Net loss 
 509 
 3.64 
 
 
 12 
 the end of 
 501 
 3.58 
 
 
 13 
 cts a share 
 490 
 3.50 
 
 
 14 
 the company 's 
 476 
 3.40 
 
 
 15 
 the company said 
 426 
 3.05 
 
 
 16 
 is expected to 
 426 
 3.05 
 
 
 17 
 said in a 
 407 
 2.91 
 
 
 18 
 Corp said it 
 392 
 2.80 
 
 
 19 
 Nine mths Shr 
 370 
 2.65 
 
 
 20 
 cts Oper net 
 363 
 2.60 
 
 
 
 
 
 Report based on word tokens 
 


 
 Ngram length: 3 
 


 
 Ngrams containing punctuation tokens excluded 
 


 
 Normalized Frequency is per 10,000 tokens 
 


 
 Total unique ngrams: 702,051 
 


 
 Total ngrams: 1,128,352 
 


 
 Showing 20 rows 
 


 
 Page 1 of 35103

In [ ]:
ngrams_reuters.ngram_frequencies(ngram_length = 3, case_sensitive = False, show_document_frequency = True).display()

Ngram Frequencies 
 
 
 Reuters Corpus 
 
 
 Rank 
 Ngram 
 Frequency 
 Normalized Frequency 
 Document Frequency 
 
 
 
 
 1 
 the company said 
 1,173 
 8.39 
 911 
 
 
 2 
 mln dlrs in 
 795 
 5.68 
 549 
 
 
 3 
 cts vs loss 
 665 
 4.75 
 474 
 
 
 4 
 said it has 
 636 
 4.55 
 586 
 
 
 5 
 mln avg shrs 
 620 
 4.43 
 450 
 
 
 6 
 pct of the 
 608 
 4.35 
 508 
 
 
 7 
 the united states 
 603 
 4.31 
 391 
 
 
 8 
 qtr net shr 
 574 
 4.10 
 573 
 
 
 9 
 dlrs a share 
 546 
 3.90 
 375 
 
 
 10 
 inc said it 
 523 
 3.74 
 521 
 
 
 11 
 the company 's 
 518 
 3.70 
 406 
 
 
 12 
 cts net loss 
 517 
 3.70 
 389 
 
 
 13 
 the end of 
 501 
 3.58 
 384 
 
 
 14 
 cts a share 
 494 
 3.53 
 290 
 
 
 15 
 is expected to 
 429 
 3.07 
 381 
 
 
 16 
 corp said it 
 412 
 2.95 
 410 
 
 
 17 
 nine mths shr 
 412 
 2.95 
 412 
 
 
 18 
 said in a 
 407 
 2.91 
 393 
 
 
 19 
 the bank of 
 380 
 2.72 
 304 
 
 
 20 
 billion dlrs in 
 373 
 2.67 
 235 
 
 
 
 
 
 Report based on word tokens 
 


 
 Ngram length: 3 
 


 
 Ngrams containing punctuation tokens excluded 
 


 
 Normalized Frequency is per 10,000 tokens 
 


 
 Total unique ngrams: 684,778 
 


 
 Total ngrams: 1,128,352 
 


 
 Showing 20 rows 
 


 
 Page 1 of 34239

In [ ]:
#| hide
# congress = Corpus().load(f'{save_path}us-congressional-speeches-subset-100k.corpus')
# report_congress = Ngrams(congress)
# congress._init_token_arrays()

In [ ]:
#| hide
# bnc = Corpus().load(f'{save_path}bnc.corpus')
# report_bnc = Ngrams(bnc)
# %time report_bnc.ngram_frequencies()

In [ ]:
#| hide
# # testing for optimisations
# for i in range(0, 7):
#     count = 10 ** i
#     print(f'Count: {count}')
#     token, token_frequency = bnc.vocab.filter(
#         pl.col('frequency_lower') <= count
#     ).select(pl.col('token'), pl.col('frequency_lower')).head(1).collect().rows()[0]
#     print(f'Token: {token}, Frequency: {token_frequency}')
#     start_time = time.time()
#     report_bnc.ngrams(token)
#     time_taken = time.time() - start_time
#     print(f'Time taken: {time_taken:.2f} seconds')
#     # time per token frequency
#     print(f'Time per token frequency: {time_taken / token_frequency:.6f} seconds')
#     print('---')   

In [ ]:
#| hide
# %time report_congress.ngram_frequencies(ngram_length = 2, case_sensitive = False, page_current = 1).display()
# report_congress.ngram_frequencies(ngram_length = 2, case_sensitive = False, page_current = 1).display()

In [ ]:
#| hide
# %time report_congress.ngrams('senator', ngram_length = 2, ngram_token_position = 'RIGHT', page_size = 10).display()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()